In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from clda import ClassificationModel

In [2]:
import sys 

sys.path.append('../../../')

from bin.main import get_path 
from main import Manager 


m = Manager(get_path('../../../'))

2025-05-03 15:43:03,199 - bin.options.optgd.db_connect - INFO - Options DB Initialized
2025-05-03 15:43:03,200 - bin.options.optgd.db_connect - WARNING - You currently have 8 connections open.
2025-05-03 15:43:03,201 - INFO - PriceDB Initialized successfully at 2025-05-03 15:43:03.201294
2025-05-03 15:43:03,201 - bin.price.db_connect - INFO - PriceDB Initialized successfully at 2025-05-03 15:43:03.201294
2025-05-03 15:43:03,201 - INFO - Established 3 database connections
2025-05-03 15:43:03,201 - bin.price.db_connect - INFO - Established 3 database connections
2025-05-03 15:43:03,392 - bin.alerts.options_alerts - INFO - Notifications instance initialized.


In [3]:
data = m.Pricedb.model_preparation('spy', daily = True, start_date = "2022-12-31")

In [21]:
X = data['X']
y = data['y']
y = pd.Series(np.where(y > 0, 1, 0), name='target', index = X.index)
X_new = data['test_data'].to_frame().T
print(X.shape, y.shape, X_new.shape)

(553, 43) (553,) (1, 43)


In [26]:
# Initialize and run the model
model = ClassificationModel(
    X=X,
    y=y,
    numerical_cols=X.columns.tolist(),
    verbose = 0
)

2025-05-03 15:47:40,727 - clda - INFO - ClassificationModel initialized with X shape: (553, 43)


In [27]:
model.preprocess_data()
model.train_models()


2025-05-03 15:47:42,683 - clda - INFO - Starting data preprocessing
2025-05-03 15:47:42,757 - clda - INFO - PCA data preparation completed with 3 components
2025-05-03 15:47:42,758 - clda - INFO - Data preprocessing completed. Training set shape: (387, 43)
2025-05-03 15:47:42,762 - clda - INFO - Evaluating LDA
2025-05-03 15:47:42,774 - clda - INFO - LDA evaluation completed. Accuracy: 0.560
2025-05-03 15:47:42,806 - clda - INFO - Evaluating QDA
2025-05-03 15:47:42,820 - clda - INFO - QDA evaluation completed. Accuracy: 0.651
2025-05-03 15:47:42,825 - clda - INFO - Evaluating Naive Bayes
2025-05-03 15:47:42,842 - clda - INFO - Naive Bayes evaluation completed. Accuracy: 0.584
2025-05-03 15:47:43,262 - clda - INFO - Best KNN neighbors: 3
2025-05-03 15:47:43,262 - clda - INFO - Evaluating KNN
2025-05-03 15:47:43,270 - clda - INFO - KNN evaluation completed. Accuracy: 0.536
2025-05-03 15:47:43,469 - clda - INFO - Best PCA KNN neighbors: 9
2025-05-03 15:47:43,470 - clda - INFO - Evaluating 

In [28]:
# Display results
results = model.get_results()
print("\nModel Performance Results:")
print(results)



Model Performance Results:
                     Accuracy Precision Sensitivity Specificity
LDA                  0.560241  0.690722    0.609091    0.464286
QDA                  0.650602  0.720339    0.772727    0.410714
Naive Bayes          0.584337  0.699029    0.654545    0.446429
KNN                  0.536145  0.681319    0.563636    0.482143
PCA KNN              0.512048  0.652632    0.563636    0.410714
Logistic Regression  0.560241  0.690722    0.609091    0.464286
Neural Network       0.512048  0.662921    0.536364    0.464286


In [29]:
model.predict_new_data(X_new)

2025-05-03 15:47:47,876 - clda - INFO - Predicting on new data with shape: (1, 43)
2025-05-03 15:47:47,896 - clda - INFO - Predictions generated for 1 samples


,LDA,QDA,Naive Bayes,KNN,PCA KNN,Logistic Regression,Neural Network
0,1,1,1,1,1,1,1


In [ ]:
model.models['LDA'].predict(X)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,